In [1]:
import csv 

MNIST = "D:/Data/mnist/mnist_train.csv"

mnist = open(MNIST, 'r')
data = csv.reader(mnist, delimiter=',')
labels = [int(row[0]) for row in data]


In [2]:
mnist = open(MNIST, 'r')
data = csv.reader(mnist, delimiter=',')
train = [[float(row[x]) / 255 for x in range(1, len(row))] for row in data]


In [3]:
# Convert labels to one-hot encoding

import numpy as np

num_labels = len(labels)
num_classes = 10

index_offset = np.arange(num_labels) * num_classes
labels_one_hot = np.zeros((num_labels, num_classes))
labels_one_hot.flat[index_offset + labels] = 1

In [7]:
print (labels_one_hot.shape)

(60000, 10)


In [5]:
images = np.array(train)

In [6]:
print (images.shape)

(60000, 784)


In [8]:
# Split data to training data and test data

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels_one_hot, test_size = 0.2)

print(X_train.shape[0])
print (X_test.shape[0])

48000
12000


In [9]:
# Build model
# Init weight
import tensorflow as tf

def weight_variable(shape):
    initial = tf.truncated_normal(shape = shape, stddev=0.1)
    return tf.Variable(initial)

# Init bias
def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

# Convolution layer
def conv2d(x, W, stride, padding):
    return tf.nn.conv2d(x, W, strides = [1, stride, stride, 1], padding = padding)

# Pooling layer
def max_pool(x, size, stride, padding):
    return tf.nn.max_pool(x, ksize = [1, size, size, 1], strides = [1, stride, stride, 1], padding = padding)
# input
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])


In [10]:
# Layer 1
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1, 28, 28, 1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, 1, 'SAME') + b_conv1)
h_pool1 = max_pool(h_conv1, 2, 2, 'SAME')

# Layer 2
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, 1, 'SAME') + b_conv2)
h_pool2 = max_pool(h_conv2, 2, 2, 'SAME')

# Layer fc
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Output Layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


In [11]:
# Training 

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

start = 0
batch_sz = 48

saver = tf.train.Saver()


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        X_batch = X_train[start: start + batch_sz,:]
        y_batch = y_train[start: start + batch_sz,:]
        start = start + batch_sz
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x: X_batch, y_: y_batch, keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))
        train_step.run(feed_dict={x: X_batch, y_: y_batch, keep_prob: 0.5})
    save_path = saver.save(sess, "../MnistModel/cnn.bin")


step 0, training accuracy 0.0416667
step 100, training accuracy 0.875
step 200, training accuracy 0.895833
step 300, training accuracy 0.875
step 400, training accuracy 0.958333
step 500, training accuracy 0.9375
step 600, training accuracy 0.958333
step 700, training accuracy 0.958333
step 800, training accuracy 1
step 900, training accuracy 1
